<img src="images/logo.png" width="200">

# KogSys-KI-B - Assignment 2

### Adversarial Search, Constraint Satisfaction Problems

_Abgabefrist: **15.06.2025**_

---



#### Abgabe Informationen

Laden Sie Ihre Lösung über den VC-Kurs hoch. Bitte laden Sie **ein Zip-Archiv** pro Gruppe hoch. Dieses muss enthalten:

- Ihre Lösung als **Notebooks** (pro Gesamtaufgabe eine `.ipynb`-Datei)
- Ein Ordner mit dem Namen **images**, der alle Ihre Bilder enthält, falls Sie welche verwendet haben (halten Sie die Bildgrößen relativ klein)

Ihr Zip-Archiv sollte wie folgt benannt werden:

```
assignment_<Assignmentnummer>_solution_<Gruppennummer>.zip
```

In dieser Aufgabe können Sie insgesamt **30 Punkte** erreichen. Von diesen Punkten werden **3 Bonuspunkte** für die Prüfung wie folgt berechnet:

| **Points in Assignment** | **Bonus Points for Exam** |
| :----------------------: | :-----------------------: |
|            30            |             3             |
|            25            |            2.5            |
|            20            |             2             |
|            15            |            1.5            |
|            10            |             1             |
|            5             |            0.5            |

<div class='alert alert-block alert-danger'>

##### **Wichtige Hinweise**

1. **Diese Aufgabe wird benotet. Sie können Bonuspunkte für die Prüfung erwerben.**
2. **Wenn offensichtlich ist, dass eine Aufgabe von einer anderen Quelle kopiert wurde und keine eigenständige Arbeit geleistet wurde, werden keine Bonuspunkte vergeben. Bitte formulieren Sie alle Antworten in Ihren eigenen Worten!**
3. **Falls LLMs (wie ChatGPT oder Copilot) zur Erstellung Ihrer Einreichung verwendet wurden, geben Sie dies bitte gemäß den gängigen wissenschaftlichen Praktiken an. Siehe auch die [KI Policy im VC-Kurs](https://vc.uni-bamberg.de/mod/page/view.php?id=1980835)**

### Setup

Um euer Assignment aufzusetzen, müsst ihr die notwendigen pakete installieren, welche in der Datein `requirements.txt` gelistet sind. Dies könnt ihr machen, indem ihr die folgende Zelle ausführt.

In [22]:
# Installiert die benötigten Pakete mit dem akutell ausgewählten Python-Interpreter
%pip install -U -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Bibliotheks-Imports

**_Import von Bibliotheken._** In der folgenden Zelle werden einige wichtige Bibliotheken importiert. Dies soll hier kurz erläutert werden. Verwenden Sie keine anderen Drittanbieterbibliotheken.

- `dataclasses.dataclass`: Einfache Erstellung von Immuatablen Klassen.
- `dataclasses.field`: Funktionen verwenden um Standart-Werte in `@dataclasses` festzulegen.
- `random`: Zufallszahlen erhalten.
- `typing.List`: Wird für Typanmerkungen in den Methodenspezifikationen benötigt.
- `connect4.Player` und `connect4.GameEngine`: Importiert den von uns vorgegebenen, ausgelagerten, Code. Dieser muss nicht selbst nachvollzogen werden.

**_In der nächsten Codezelle muss nichts geändert werden._**

In [23]:
# Python Core libraries
from dataclasses import dataclass, field
import random
from typing import List, Optional

# Connect4 Game Engine
from connect4 import Player, Connect4GameEngine, BoardType

## Aufgabe 1 | Adversarial Search am Beispiel "4 Gewinnt"

_Für insgesamt <mark>15</mark> Punkte_

In dieser Aufgabe wollen wir [4 Gewinnt](https://de.wikipedia.org/wiki/Vier_gewinnt) gegen unseren Computer spielen. Hierzu soll ein aus der Vorlesung bekannter Algorithmus für _Adversarial Games_ implementiert werden.

Betrachtet und versteht den gegebenen Code jedoch zuerst!

#### Spielfeld aufbauen

Im Folgenden definieren wir eine Funktion, und zwei Konstanten, welche es uns erlauben unser Spielfeld als zweidimensionale Liste zu erstellen.
Somit haben wir eine Matrix der Form:

```python
[
# Spalte  0     1     2     3     4     5     6  
        [None, None, None, None, None, None, None],  # Zeile 0
        [None, None, None, None, None, None, None],  # Zeile 1
        [None, None, None, None, None, None, None],  # Zeile 2
        [None, None, None, None, None, None, None],  # Zeile 3
        [None, None, None, None, None, None, None],  # Zeile 4
        [None, None, None, None, None, None, None]   # Zeile 5
]
```

***In der nächsten Codezelle muss nichts geändert werden.***

In [24]:
# The size of the game grid.
GRID_ROWS = 6
GRID_COLUMNS = 7


def create_grid() -> List[List[Optional[Player]]]:
    """
    Create a new grid of the correct size.

    :returns grid: The new grid.
    """
    return [[None] * GRID_COLUMNS for _ in range(GRID_ROWS)]

#### **(02.1.1)** Spielfeld Logik

_Für <mark>3</mark> Punkte_

Da wir nun wissen wie wir das Spielfeld speichern, wollen wir dies in einer Klasse umsetzen. Diesen Teil haben wir euch schon vorgegeben. Eure Aufgabe ist es nun, die beiden fehlenden Methoden `valid_moves` und `get_winner` zu implementieren. Zusätzlich dürft ihr natürlich weitere Funktionen implementieren, jedoch die vorgegebene Funktion `drop_in_column` nicht ändern!

##### Methode `valid_moves`
Diese Methode soll eine Liste aller Spalten zurückgeben, in die aktuell noch ein Spielstein geworfen werden kann. Vergesst nicht, das unser Spielfeld mit einem _Zero-Index_ arbeitet!

#### Methode `is_game_over`
Diese Methode soll überprüfen, ob das Spiel vorbei ist, weil es entweder bereits einen Gewinner gibt, oder weil kein Zug mehr möglich ist..

#### Methode `get_winner`
Diese Methode soll den Gewinner des aktuellen Spiels zurückgeben, oder eben `None`, falls es noch keinen gibt.

<details>

<summary>Tipp</summary>

Eine neue Hilfsfunktion kann euch die Arbeit erleichtern!

</details>


In [29]:
@dataclass(frozen=True)
class Board(BoardType):
    """The game board."""

    grid: List[List[Optional[Player]]] = field(default_factory=create_grid)

    def drop_in_column(self, player: Player, column: int) -> "Board":
        """
        Drop a disc for the given player in the given column.

        :param player: The player who should drop a disk.
        :param column: The column in which to drop a disk.
        :returns board: The resulting board.
        """
        for row in reversed(range(len(self.grid))):
            if not self.grid[row][column]:
                new_grid = [list(r) for r in self.grid]

                new_grid[row][column] = player

                return Board(grid=new_grid)

        raise ValueError(f"Could not drop in '{column}' for '{player.name}', is it full?")

    def valid_moves(self) -> List[int]:
        return [column for column in range(GRID_COLUMNS) if self.grid[0][column] is None]
    
    def is_game_over(self) -> bool:
        # einfach getwinner aufrufen und prüfen, ob der Wert None ist
        # TODO ist safe nicht so gedacht, vllt andere lösung finden
        return self.get_winner() is not None

    def get_winner(self) -> Optional[Player]:
        """
        Get the currently winning player.

        :returns winner: The currently winning player or None if there is no winner (game not over / tie).
        """
        # TODO Definitiv nicht optimal, WIP 😭😭😭😭
        # Zeitkomplexität O(4 Generationen)
        
        """Anderer Ansatz, aber noch nicht implementiert:
        Eine Check funktion, die über eine beliebige (auch nicht rechteckige)  Matrix iteriert

        Auf folgende Matritzen anwenden (und Abbrechen wenn halt wer gewonnen hat):
            Normale Matrix
            Matrix transformieren,
            Matrix diagonal transformieren 
            Matrix diagonal transformieren (umgekehrt)
        
        Annahme: Es gibt immer nur einen Gewinner
        """
        horizontal_transform = [[None] * GRID_COLUMNS for _ in range(GRID_ROWS)]
        for row in range(GRID_ROWS):
            for column in range(GRID_COLUMNS - 3):
                horizontal_transform[row][column] = self.grid[row][column]


        diagonal_transform1 = [[self.grid[row + i][column + i] for i in range(4)] for row in range(GRID_ROWS - 3) for column in range(GRID_COLUMNS - 3)]
        diagonal_transform2 = [[self.grid[row + i][column - i] for i in range(4)] for row in range(GRID_ROWS - 3) for column in range(3, GRID_COLUMNS)]
        checks = [self.grid, horizontal_transform, diagonal_transform1, diagonal_transform2]

        for matrix in checks:
            winner = self.check_matrix(matrix)
            if winner is not None:
                return winner
        return None

    
    def check_matrix(self, matrix: List[List[Optional[Player]]]) -> Optional[Player]:
        """
        Check a given matrix for a winner.

        :param matrix: The matrix to check.
        :returns winner: The winning player or None if there is no winner.
        """
        
        for column in range(len(matrix) - 3):
            for row in range(len(matrix[0])):
                if (matrix[row][column] is not None and
                    matrix[row][column] == matrix[row][column + 1] == 
                    matrix[row][column + 2] == matrix[row][column + 3]):
                    return matrix[row][column]
        return None
    

# Testen:
# if __name__ == "__main__":
#     # Play a random game
#     board = Board()
#     print(board.valid_moves())
#     print(board.is_game_over())

#     while not board.is_game_over() and board.valid_moves():
#         column = random.choice(board.valid_moves())
#         player = Player(name=f"Player {random.randint(1, 2)}", next_move=None)
#         board = board.drop_in_column(player, column)
#         print(f"{player.name} dropped in column {column}.")
#         print(board.grid)

    


#### **(02.1.2)** KI Gegner

_Für <mark>12</mark> Punkte_

Nun haben wir ein funktionsfähiges Spielfeld. Jetzt sollten wir uns darauf konzentrieren einen Gegner zu bauen.

Die aktuelle Implementierung von `ai_next_move` wählt eine zufälligen erlaubte Spalte aus, dies macht den Gegner jedoch ziemlich einfach zu besiegen. Implementiert deshalb einen Algorithmus _aus der Vorlesung_ eurer Wahl. Vergesst nicht, dass auch die Spalten mit einem _Zero-Index_ arbeiten, sprich wenn in Spalte 1 ein Spielstein geworfen werden soll, muss eure Funktion `0` zurückgeben.

Natürlich dürft ihr auch wieder weitere Funktionen definieren.

**Wichtig:** Vier-Gewinnt ist möglicherweise mit den Algorithmen aus der Vorlesung nicht sehr performant. Macht entsprechend Anpassungen an eurer Implementation, um schneller Ergebnisse zu erhalten.

Bitte erklärt euren Ansatz und zusätzliche Anpassungen
 
<details>

<summary>Tipps</summary>

Ihr erhaltet in der Funktion, zusätzlich zum Spielfeld noch den `ai_player`, welcher einen guten Zug machen sollte. Auch erhaltet ihr den `opponent`, für welchen der Zug schlecht sein sollte.

Diese Spieler werden im `grid` des `Board`s gespeichert, sprich, wenn `board.grid[0][0] == ai_player` gilt, hat der KI-Spieler einen Spielstein im oberen linken Feld.

Eine mögliche Anpassung ist, dem Algorithmus ein zusätzliches Argument `depth` zu übergeben, welches die Rekursionstiefe limitiert. Wählt hierfür einen sinnvollen Wert aus.

Wenn euer Algorithm Tiefen-Limitiert ist, wie entscheiden sie den Score für einen nicht-Endzustand?

</details>

> 
> Hier kann eure Antwort eingegeben werden.
> 

In [26]:
def ai_next_move(board: BoardType, ai_player: Player, opponent: Player) -> int:
    """
    Get the next column the AI should put a disk in.

    :param ai_player: The instance of the AI player.
    :param opponent: The instance of the other player.
    :param board: The current game board.
    :returns column: The column in which to put a disk. This is zero indexed from left to right!
    """

    # denke hier bietet sich ein Minimax an, weil es sich um ein 1v1 Spiel handelt 
    # werte sind -1, 0, 1. 1 gewinnt ai, 0 unentschieden, -1 verliert ai
    # Aber vorsicht hinweis oben: Nicht performant: Verbesserunsideen? TODO

    valid_moves = board.valid_moves()

    # You can safely assume there are possible moves.
    assert len(valid_moves) > 0, "No moves possible."

    print(f"{ai_player.name} is chosing a random  field...")
    return random.choice(valid_moves)

### Spielen

Nun könnt ihr euren Algorithmus testen - und gegen diesen Vier gewinnt spielen. Führt die folgende Code-Zelle aus um das Spiel zu starten. Natürlich dürft ihr die Code-Zelle auch anpassen um z.B. weitere Spieler hinzuzufügen oder die Spieler auszutauschen.

<details>
  <summary>Spieler erstellen und austauschen</summary>

  - **KI-Spieler**: Erstelle mit `Player.ai(name="Name", move=ai_next_move)`:
    ```python
    AI_PLAYER_1 = Player.ai(name="AI 1", move=ai_next_move)
    ```

    Die Funktion `ai_next_move` kann natürlich auch durch eine andere Funktion ersetzt werden, sie muss lediglich die korrekte Signatur, wie durch `ai_next_move` vorgegeben, haben.

  - **Menschlicher Spieler**: Erstelle mit `Player.human(name="Name")`:
    ```python
    HUMAN_PLAYER_1 = Player.human(name="Human 1")
    ```
  - **Spieler austauschen**: Ändere `player0` und `player1` in `GameEngine`:
    ```python
    game = GameEngine(board=Board(), player0=HUMAN_PLAYER_1, player1=AI_PLAYER_1)
    ```

</details>

<details>
  <summary>Spielfeld austauschen</summary>

Standardmäßig startet das Spiel mit einem leeren Spielfeld der Größe `GRID_ROWS x GRID_COLUMNS`. Falls ihr gezielt bestimmte Spielsituationen testen möchtet, könnt ihr mit dem vorausgefüllten `Board` `EXAMPLE_BOARD` starten, tauscht hierzu einfach in der nächsten Zelle den Wert für `board=` aus.

_Natürlich dürft ihr das `EXAMPLE_BOARD` auch ändern oder weitere Beispiele bauen. Um den Suchraum zu verkleinern könnt ihr auch ein kleineres Spielfeld übergeben._

</details>

<br>

<div class="alert-warning" style="padding: 0.5rem">
Das Spiel funktioniert nur, wenn ihr die Aufgabe 02.1.1 korrekt bearbeitet habt. Solltet ihr daher eine Fehlermeldung erhalten, prüft ob im Stacktrace irgendwo auf euren Code verwiesen wird!
</div>


In [27]:
AI_PLAYER_1 = Player.ai(name="AI 1", move=ai_next_move)
AI_PLAYER_2 = Player.ai(name="AI 2", move=ai_next_move)

HUMAN_PLAYER_1 = Player.human(name="Human 1")
HUMAN_PLAYER_2 = Player.human(name="Human 2")

EMPTY_BOARD = Board()
EXAMPLE_BOARD = Board(grid=[
    [None,        None,           None,           None, None, None, None], 
    [None,        None,           None,           None, None, None, None], 
    [None,        None,           None,           None, None, None, None], 
    [None,        None,           HUMAN_PLAYER_1, None, None, None, None], 
    [None,        AI_PLAYER_1,    HUMAN_PLAYER_1, None, None, None, None],
    [AI_PLAYER_1, HUMAN_PLAYER_1, AI_PLAYER_1,    None, None, None, None],
])

In [30]:
# You are allowed to change the values for "board", "player0" and "player1" so you can challenge algorithms against each other or to play with your friends!
connect4 = Connect4GameEngine(
    board=EMPTY_BOARD, 
    player0=HUMAN_PLAYER_1,
    player1=AI_PLAYER_1
)

connect4.start()

Output()

Output()

IndexError: list index out of range


#### **(02.1.3, optional)** 4-Gewinnt Turnier

*Für bis zu <mark>10 zusätzliche</mark> Punkte*

<details>
<summary>Hinweis zu den Zusatzpunkten</summary>
Ihr könnt bis zu 10 zusätzliche Punkte erhalten, wenn ihr am Turnier teilnehmt. Die Zusatzpunkte werden basierend auf der Leistung eurer KI im Turnier vergeben.

Die Punkte werden zu eurer Gesamtpunktzahl der Abgaben hinzugefügt, jedoch kann das Maximum von 10 Bonuspunkten für die Prüfung nicht überschritten werden. Das bedeutet: Wenn ihr bereits alle 100 Punkte aus den Abgaben gesammelt habt, ändert sich nichts. Wenn ihr jedoch z. B. nur 80 Punkte gesammelt habt, könnt ihr bis zu 9 Bonuspunkte für die Prüfung erhalten.

</details>

Das Turnier findet am **17. Juni 2025** während der Übung statt. Um teilzunehmen, müsst ihr im folgenden Markdown-Abschnitt einen Teamnamen eintragen. (Der Teamname dient der Anonymität und stellt sicher, dass ihr teilnehmen möchtet.)

Die Punktevergabe richtet sich nach der Leistung eurer KI im Turnier. Je höher eure Gewinnrate, desto mehr Punkte erhaltet ihr. Die Formel zur Berechnung der Punkte lautet:

```python
Points_i = round(10 * (W_i / W_max))
```

wobei `W_i` das Gewinnverhältnis eures KI-Agenten ist und `W_max` das beste Gewinnverhältnis aller KI-Agenten im Turnier.


## Aufgabe 2 - Bedingungserfüllungsprobleme (CSP) am Beispiel "Hashiwokakero"

_Für insgesamt <mark>15</mark> Punkte_

<div class="alert-warning" style="padding: 0.5rem">

Diese Aufgabe wird in der separaten Datei `assignment_2_task_2_german.ipynb` bearbeitet.

</div>

Diese Aufgabe implementiert einen **Constraint Satisfaction Problem (CSP) Solver** für das [Hashiwokakero Puzzel-Spiel](https://de.wikipedia.org/wiki/Hashiwokakero). Ihre Aufgabe ist es, die CSP-Solver Implementierung vervollständigen.

Ihr könnt das Spiel selbst hier ausprobieren: https://de.hashi.info/.

## CSP Formulierung:
* **Variables (X)**: Potentielle Brücken zwischen benachbarten Inseln
* **Domains (D)**: Anzahl Brücken (0, 1, oder 2) zwischen Inselpaaren
* **Constraints (C)**:
  - Jede Insel muss exakt die durch ihre Zahl angegebene Anzahl Brücken haben
  - Brücken können sich nicht kreuzen
  - Brücken können nur horizontal oder vertikal verlaufen
  - Alle Inseln müssen verbunden sein um einen einzigen Graphen zu bilden

